Obtain the list of postal codes of Canada and transform to a pandas dataframe

Installing packages:

In [2]:
try:
    print("Installing BeautifulSoup4...\n")
    !conda install -c conda-forge beautifulsoup4 --yes
    print("BeautifulSoup4 has been successfully installed!\n")
except:
    print("ERROR: could not install BeautifulSoup4!\n")
try:
    print("Installing lxml...\n")
    !conda install -c conda-forge lxml --yes
    print("lxml has been successfully installed!\n")
except:
    print("ERROR: could not install lxml!\n")
try:
    print("Installing ProgressBar...\n")
    !conda install -c conda-forge ProgressBar2 --yes
    print("ProgressBar has been successfully installed!\n")
except:
    print("ERROR: could not install ProgressBar!\n")
try:
    print("Installing GeoPy...\n")
    !conda install -c conda-forge geopy --yes
    print("GeoPy has been successfully installed!\n")
except:
    print("ERROR: could not install GeoPy!\n")
try:
    print("Installing Folium...\n")
    !conda install -c conda-forge folium=0.5.0 --yes
    print("Folium has been successfully installed!\n")
except:
    print("ERROR: could not install Folium!\n")
try:
    print("Installing HTML5LIB...\n")
    !conda install -c conda-forge html5lib --yes
    print("HTML5LIB has been successfully installed!\n")
except:
    print("ERROR: could not install HTML5LIB!\n")

Installing BeautifulSoup4...

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.8.1       |           py36_0         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following packages will be UPDATED:

    beautifulsoup4:  4.7.1-py36_1      --> 4.8.1-py36_0          conda-forge
    ca-certificates: 2019.11.27-0      --> 2019.11.28-hecc5488_0 conda-forge
    certifi:         2019.11.28-py36_0 -->

Importing libraries:

In [3]:
try:
    print("Importing libraries...\n")
    from progressbar import ProgressBar
    from bs4 import BeautifulSoup as bts # library for web scraping
    import numpy as np # library to handle data in a vectorized manner
    import pandas as pd # library for data analysis
    from pandas.io.json import json_normalize
    import matplotlib.cm as cm
    import matplotlib.colors as colors
    import requests # library to handle requests
    from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
    import matplotlib as mp # library for visualization
    from sklearn.cluster import KMeans # import k-means from clustering stage
    from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
    import folium # map rendering library
    import lxml
    import lxml.html as lh
    import re
    from time import sleep
    print("All libraries imported successfully!\n")
except:
    print("ERROR: Could not import all libraries!\n")

%matplotlib inline

Importing libraries...

All libraries imported successfully!



Obtaining the data from the wiki site:

In [44]:
try:
    print("Requesting source...")
    source  = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
    print("Source successfully requested!")
except:
    print("ERROR: could not request the source.\n")
try:
    print("Creating beautifulsoup object from source data...")
    soup = bts(source,'lxml')
    print("Successfully created beautifulsoup object from source data!")
except:
    print("ERROR: could not create beautifulsoup object from source data!")
try:
    print("Finding all tables in source code...")
    table = soup.findAll('table',class_='wikitable sortable')
    print("Tables were successfully found in source code!")
    print("Number of tables in site:",len(table))
except:
    print("ERROR: could not find any table in source code!")
table = table[0]
print("Getting the table text list")
table_text = table.tbody.text
table_text = re.sub("\[.*?\]", "",table_text)
table_list = table_text.split('\n')
del table_list[-1]
print("Table text list obtained successfully.")
print("Getting the table column headers")
table_columns = table_list[0:5]
table_columns[0] = 'NotRequired1'
table_columns[4] = 'NotRequired2'
print("Table column headers: ", table_columns[1:4])
print("Number of data in table: ",len(table_list[5:]))
if (len(table_list[5:]) % 5 == 0):
    table_data = np.array(table_list[5:]).reshape(len(table_list[5:]) // 5,5)
else:
    print("Number of table elements is incorrect!")
print("Preparing the required dataframe and table.")
dataframe =  pd.DataFrame(np.nan_to_num(table_data),columns = table_columns)
print("Table shape as per site data: ", dataframe.shape)
print(dataframe.head(5))
dataframe1 = dataframe.drop(['NotRequired1','NotRequired2'],axis=1)
dataframe1 = dataframe1.reset_index(drop=True)
print("Table shape after removing not required blank columns: ",dataframe1.shape)
print(dataframe1.head(5))

Requesting source...
Source successfully requested!
Creating beautifulsoup object from source data...
Successfully created beautifulsoup object from source data!
Finding all tables in source code...
Tables were successfully found in source code!
Number of tables in site: 1
Getting the table text list
Table text list obtained successfully.
Getting the table column headers
Table column headers:  ['Postcode', 'Borough', 'Neighbourhood']
Number of data in table:  1435
Preparing the required dataframe and table.
Table shape as per site data:  (287, 5)
  NotRequired1 Postcode           Borough     Neighbourhood NotRequired2
0                   M1A      Not assigned      Not assigned             
1                   M2A      Not assigned      Not assigned             
2                   M3A        North York         Parkwoods             
3                   M4A        North York  Victoria Village             
4                   M5A  Downtown Toronto      Harbourfront             
Table sha

Removing the Boroughs with 'Not assigned' values:

In [35]:
dataframe2 = dataframe1[dataframe1.Borough!="Not assigned"]
dataframe2 = dataframe2.reset_index(drop=True)
print("Table shape after removing Boroughs with Not assigned values: ",dataframe2.shape)
print(dataframe2.head(20))


Table shape after removing Boroughs with Not assigned values:  (210, 3)
   Postcode           Borough     Neighbourhood
0       M3A        North York         Parkwoods
1       M4A        North York  Victoria Village
2       M5A  Downtown Toronto      Harbourfront
3       M6A        North York  Lawrence Heights
4       M6A        North York    Lawrence Manor
5       M7A      Queen's Park      Not assigned
6       M9A  Downtown Toronto      Queen's Park
7       M1B       Scarborough             Rouge
8       M1B       Scarborough           Malvern
9       M3B        North York   Don Mills North
10      M4B         East York  Woodbine Gardens
11      M4B         East York     Parkview Hill
12      M5B  Downtown Toronto           Ryerson
13      M5B  Downtown Toronto   Garden District
14      M6B        North York         Glencairn


Updating Neighbourhoods with values as 'Not assigned' to the corresponding Borough value:

In [79]:
#print(dataframe2.loc[dataframe2['Neighbourhood'] == 'Not assigned','Neighbourhood'])
#print(dataframe2.loc[dataframe2['Neighbourhood'] == 'Not assigned','Borough'])
dataframe2.loc[dataframe2['Neighbourhood'] == 'Not assigned','Neighbourhood'] = dataframe2['Borough']
dataframe2.reset_index()
print("Shape of table after updating the neighbourhood values: ",dataframe2.shape)
print(dataframe2.head(20))

Shape of table after updating the neighbourhood values:  (210, 3)
   Postcode           Borough     Neighbourhood
0       M3A        North York         Parkwoods
1       M4A        North York  Victoria Village
2       M5A  Downtown Toronto      Harbourfront
3       M6A        North York  Lawrence Heights
4       M6A        North York    Lawrence Manor
5       M7A      Queen's Park      Queen's Park
6       M9A  Downtown Toronto      Queen's Park
7       M1B       Scarborough             Rouge
8       M1B       Scarborough           Malvern
9       M3B        North York   Don Mills North
10      M4B         East York  Woodbine Gardens
11      M4B         East York     Parkview Hill
12      M5B  Downtown Toronto           Ryerson
13      M5B  Downtown Toronto   Garden District
14      M6B        North York         Glencairn
15      M9B         Etobicoke        Cloverdale
16      M9B         Etobicoke         Islington
17      M9B         Etobicoke      Martin Grove
18      M9B         Et

In [81]:
dataframe3 = dataframe2.groupby(['Postcode', 'Borough']).agg(lambda x: ", ".join(sorted(set(x))))
dataframe3 = dataframe3.reset_index()
print("Table shape after grouping and joining neighborhoods in same postal code: ", dataframe3.shape)
print(dataframe3.head(30))
print("Table prepared successfully!")    


Table shape after grouping and joining neighborhoods in same postal code:  (103, 3)
   Postcode      Borough                                      Neighbourhood
0       M1B  Scarborough                                     Malvern, Rouge
1       M1C  Scarborough             Highland Creek, Port Union, Rouge Hill
2       M1E  Scarborough                  Guildwood, Morningside, West Hill
3       M1G  Scarborough                                             Woburn
4       M1H  Scarborough                                          Cedarbrae
5       M1J  Scarborough                                Scarborough Village
6       M1K  Scarborough        East Birchmount Park, Ionview, Kennedy Park
7       M1L  Scarborough                    Clairlea, Golden Mile, Oakridge
8       M1M  Scarborough    Cliffcrest, Cliffside, Scarborough Village West
9       M1N  Scarborough                        Birch Cliff, Cliffside West
10      M1P  Scarborough  Dorset Park, Scarborough Town Centre, Wexford ...
11  

In [51]:
print("Final number of rows in dataframe: ", dataframe3.shape)

Final number of rows in dataframe:  (103, 3)


Creating columns for the Latitude and Longitude for each Postal Code: 

In [53]:
dataframe3['Latitude'] = np.nan
dataframe3['Longitude'] = np.nan
dataframe3.head(20)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",NaN,NaN
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",NaN,NaN
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",NaN,NaN
3,M1G,Scarborough,Woburn,NaN,NaN
4,M1H,Scarborough,Cedarbrae,NaN,NaN
5,M1J,Scarborough,Scarborough Village,NaN,NaN
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",NaN,NaN
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",NaN,NaN
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",NaN,NaN
9,M1N,Scarborough,"Birch Cliff, Cliffside West",NaN,NaN


Gathering the Latitude and Longitude coordinates for each postal code using geolocater: 

In [58]:
pbar = ProgressBar()
geolocator = Nominatim()
location = None
while(location is None):
    for index in pbar(range(0,dataframe3['Postcode'].shape[0])):
        address = dataframe3.loc[index,'Postcode'] + ", Toronto, Ontario"
        location = geolocator.geocode(address, timeout = None)
        if (location != None):
            dataframe3.loc[index,'Latitude'] = location.latitude
            dataframe3.loc[index,'Longitude'] = location.longitude
        sleep(1)

print(dataframe3.shape)
dataframe3.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app
N/A% (0 of 103) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:7: DeprecationWarning: `timeout=None` has been passed to a geocoder call. Using default geocoder timeout. In geopy 2.0 the behavior will be different: None will mean "no timeout" instead of "default geocoder timeout". Pass geopy.geocoders.base.DEFAUL

(103, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.653963,-79.387207
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.653963,-79.387207
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",NaN,NaN
3,M1G,Scarborough,Woburn,43.644903,-79.381836
4,M1H,Scarborough,Cedarbrae,NaN,NaN


Removing addresses that have no find location details: 
Final table with location details: 

In [77]:
dataframe4 = dataframe3.dropna()
dataframe4 = dataframe4.reset_index(drop=True)
print("Table shape after removing postcodes with no location values: ",dataframe4.shape)
dataframe4.head(25)

Table shape after removing postcodes with no location values:  (24, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.653963,-79.387207
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.653963,-79.387207
2,M1G,Scarborough,Woburn,43.644903,-79.381836
3,M1W,Scarborough,L'Amoreaux West,43.644903,-79.381836
4,M2J,North York,"Fairview, Henry Farm, Oriole",43.644903,-79.381836
5,M2M,North York,"Newtonbrook, Willowdale",43.785962,-79.416031
6,M2N,North York,Willowdale South,43.644903,-79.381836
7,M3A,North York,Parkwoods,43.653963,-79.387207
8,M3C,North York,"Don Mills South, Flemingdon Park",43.732822,-79.346961
9,M3H,North York,"Bathurst Manor, Downsview North, Wilson Heights",43.756199,-79.439802
